# Using Tensorflow 2.x Transfer Learning to classify Plant seedlings ☘

### About the competition 
Many times it is very difficult to tell which plant it is from the seedlings. In this compition, we will make a ML model which can help to identify seedlings and tell which plant it is. 

### Kind of ML Problem
This kind of problem is called multi-class image classification. It's multi-class because we're trying to classify mutliple different plant seedlings. 

1. Getting data ready import dataset.
2. Visualize the Dataset.
3. Preprocess the Dataset for Tensorflow format.
4. Prepare Tensorflow Tranfer learning Model.
5. Train the model on small data.
6. Visualize the model results.
7. Train on complete data.
8. Predict on test model.

In [ ]:
import pandas as pd 
import os 
import tensorflow as tf 
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
%config Completer.use_jedi = False

In [ ]:
train_path = "../input/plant-seedlings-classification/train/"
test_path = "../input/plant-seedlings-classification"

In [ ]:
for dirpath, dirnames, filenames in os.walk("../input/plant-seedlings-classification/"): 
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

Hmm, We learnt from here two things:

There are 794 images in test folder which will be used for testing.

There are images for each class in a directory structure for training. 

## Prepare the data for Training & Validation

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_path,
                                                                 label_mode="categorical", 
                                                                 shuffle=True,
                                                                 image_size=(224,224),
                                                                batch_size = 32,
                                                                 seed=42,
                                                                validation_split=0.2,
                                                                   subset="training")


In [ ]:
valid_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_path,
                                                                 label_mode="categorical", 
                                                                 shuffle=False,
                                                                 image_size=(224,224),
                                                                batch_size = 32,
                                                                 seed=42,
                                                                validation_split=0.2,
                                                                   subset="validation")


In [ ]:
class_names = train_data.class_names
class_names

## Visualizing the dataset

In [ ]:
data = train_data.take(1)

In [ ]:
for image, label in data: 
    plt.imshow(image[0]/255.)
    plt.title(f"Target Class: {class_names[tf.argmax(label[0])]} ")


### View Images from training data set


In [ ]:
image[0], label[0]

In [ ]:
plt.figure(figsize = (20,20)) 
image, label = next(iter(train_data))
for i in range(0,25) : 
    
    ax = plt.subplot(5,5,i+1) 
    plt.imshow(image[i]/255.)
    plt.title(f"Target: {class_names[tf.argmax(label[i])]} ")
    ax.axis("off")

## Building a tranfer learning model feature extraction using the keras functional API

The sequential API is straight forward, it runs our layers in sequential order.

But the functional API gives us more flexibility with our models

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top= False)

base_model.trainable = False 

inputs = tf.keras.layers.Input(shape=(224,224,3), name='Input_Layer') 

x = base_model(inputs) 

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

outputs = tf.keras.layers.Dense(len(class_names), activation='softmax', name='output_layer')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(loss='categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(), 
             metrics=['accuracy']) 

model_history = model.fit(train_data, 
                          steps_per_epoch=len(train_data),
                          validation_data= valid_data,
                        validation_steps= len(valid_data),
                         epochs=5
                         )

In [ ]:
#Evaluate on full data
model.evaluate(valid_data)

## Visualize Model results

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
#Check our loss curves 
plot_loss_curves(model_history)

Here, from the above plot we can see our model is generalizing really well on training and validation data. Great. 

Let's try to predict and check the predictions. 

In [ ]:
prediction = model.predict(valid_data)

In [ ]:
# Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return class_names[tf.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(prediction[0])
pred_label

In [ ]:
# Create a function to unbatch a batched dataset
def unbatchify(data):
  """
  Takes a batched dataset of (image, label) Tensors and returns separate arrays
  of images and labels.
  """
  images = []
  labels = []
  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image/255.)
    labels.append(class_names[tf.argmax(label)])
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(valid_data)
val_images[0], val_labels[0]

In [ ]:
import numpy as np
def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the prediction, ground truth label and image for sample n.
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
  
  # Get the pred label
  pred_label = get_pred_label(pred_prob)
  
  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the color of the title depending on if the prediction is right or wrong
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  plt.title("{} {:2.0f}% ({})".format(pred_label,
                                      np.max(pred_prob)*100,
                                      true_label),
                                      color=color)

In [ ]:
# View an example prediction, original image and truth label
plot_pred(prediction_probabilities=prediction,
          labels=val_labels,
          images=val_images,n=33)

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plots the top 10 highest prediction confidences along with
  the truth label for sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  # Get the predicted label
  pred_label = get_pred_label(pred_prob)

  # Find the top 10 prediction confidence indexes
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]
  # Find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]
  # Find the top 10 prediction labels
  top_10_pred_labels = np.array(class_names)[top_10_pred_indexes]

  # Setup plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels)), 
                     top_10_pred_values, 
                     color="grey")
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")

  # Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=prediction,
               labels=val_labels,
               n=1)

In [ ]:
# Let's check a few predictions and their different values
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(5*2*num_cols, 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=prediction,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=prediction,
                labels=val_labels,
                n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()

## Train on full data

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_path,
                                                                 label_mode="categorical", 
                                                                 shuffle=True,
                                                                 image_size=(224,224),
                                                                batch_size = 32)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top= False)

base_model.trainable = False 

inputs = tf.keras.layers.Input(shape=(224,224,3), name='Input_Layer') 

x = base_model(inputs) 

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

outputs = tf.keras.layers.Dense(len(class_names), activation='softmax', name='output_layer')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(loss='categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(), 
             metrics=['accuracy']) 

model_history = model.fit(train_data, 
                          steps_per_epoch=len(train_data),
                         epochs=10
                         )

## Test data prediction

In [ ]:
#Prepare the Test data using Tensorflow Image Generator

datagen = ImageDataGenerator()
gen = datagen.flow_from_directory(test_path ,shuffle =False,batch_size=100,
                              target_size = (224,224),classes = ['test'])

In [ ]:
prediction = model.predict(gen)
prediction.shape

In [ ]:
class_names[tf.argmax(prediction[2])]

In [ ]:
#Read sample submission file 
sample_submission = pd.read_csv('../input/plant-seedlings-classification/sample_submission.csv')
sample_submission.head()

In [ ]:
len(sample_submission)

In [ ]:
#Prepare prediction results
predict_class = []
for pred in prediction: 
    predict_class.append(class_names[tf.argmax(pred)])

predict_class[:10]

In [ ]:
submission = pd.DataFrame({'file':sample_submission['file'],'species':predict_class})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

## What's next? 

Woah! What an effort. If you've made it this far, you've just gone end-to-end on a multi-class image classification problem.

You can try using other approaches to improve your model.

1. Data augmentation - Take the training images and manipulate (crop, resize) or distort them (flip, rotate) to create even more training data for the model to learn from. Check out the TensorFlow images documentation for a whole bunch of functions you can use on images. 

2. Fine-tuning - The model we used in this notebook was directly from TensorFlow Hub, we took what it had already learned from another dataset (ImageNet) and applied it to our own. Another option is to use what the model already knows and fine-tune this knowledge to our own dataset.


#### Please Upvote if like you my work. If you have any queries please comment. 😎